In [ ]:
from esper.prelude import *
import json

In [ ]:
videos = list(Video.objects.all())

In [ ]:
def get_durs():
    def check_meta(v):
        try:
            meta = json.load(open('/app/data/ia-metadata/{}.json'.format(v.item_name())))
            runtime = meta['metadata']['runtime']
            [h, m, s] = [int(s) for s in runtime.split(':')]
            computed_dur = v.num_frames / v.fps
            expected_dur = h * 3600 + m * 60 + s
            return (v, computed_dur, expected_dur)
        except Exception:
            return None
    return par_for(check_meta, videos, workers=16)
durs = pcache.get('durs', get_durs)

In [ ]:
ns = []
corrupted = []
for tup in durs:
    if tup is None: continue
    (v, computed_dur, expected_dur) = tup
    diff = computed_dur - expected_dur
    if abs(diff) > 100:
        ns.append(diff)
        if diff > 0:
            corrupted.append(v)
len(ns)

In [ ]:
pd.Series(ns).hist(bins=100)
plt.xlabel('Hwang duration minus Archive duration')
plt.ylabel('# of videos')

In [ ]:
for v in corrupted:
    v.corrupted = True
    v.save()